In [1]:
import pandas as pd
import numpy as np
from google.colab import files
import io

# This triggers the upload prompt. Select your 'Emergency Room.csv'
uploaded = files.upload()

Saving Hospital ER_Data.csv to Hospital ER_Data.csv


In [7]:
# 1. Load Data (Assumes filename is 'Emergency Room.csv')

try:
    df = pd.read_csv(io.BytesIO(uploaded['Emergency Room.csv']))
except KeyError:
    # If file name is different, grab the first file uploaded
    filename = list(uploaded.keys())[0]
    df = pd.read_csv(io.BytesIO(uploaded[filename]))

# 2. Header Cleaning
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')


# Convert date to datetime object
df['date'] = pd.to_datetime(df['patient_admission_date'])



# 3. Data check
print(f"Total Rows Processed: {len(df)}")
print(f"Columns Created: {list(df.columns)}")
print("Null Values Check:\n", df.isnull().sum())



Total Rows Processed: 9216
Columns Created: ['patient_id', 'patient_admission_date', 'patient_first_inital', 'patient_last_name', 'patient_gender', 'patient_age', 'patient_race', 'department_referral', 'patient_admission_flag', 'patient_satisfaction_score', 'patient_waittime', 'patients_cm', 'date']
Null Values Check:
 patient_id                       0
patient_admission_date           0
patient_first_inital             0
patient_last_name                0
patient_gender                   0
patient_age                      0
patient_race                     0
department_referral           5400
patient_admission_flag           0
patient_satisfaction_score    6699
patient_waittime                 0
patients_cm                      0
date                             0
dtype: int64


/tmp/ipython-input-3487720261.py:15: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(df['patient_admission_date'])


In [8]:
df.head()

,patient_id,patient_admission_date,patient_first_inital,patient_last_name,patient_gender,patient_age,patient_race,department_referral,patient_admission_flag,patient_satisfaction_score,patient_waittime,patients_cm,date
0,145-39-5406,20-03-2024 08:47,H,Glasspool,M,69,White,NaN,False,10.0,39,0,2024-03-20 08:47:00
1,316-34-3057,15-06-2024 11:29,X,Methuen,M,4,Native American/Alaska Native,NaN,True,NaN,27,0,2024-06-15 11:29:00
2,897-46-3852,20-06-2024 09:13,P,Schubuser,F,56,African American,General Practice,True,9.0,55,0,2024-06-20 09:13:00
3,358-31-9711,04-02-2024 22:34,U,Titcombe,F,24,Native American/Alaska Native,General Practice,True,8.0,31,0,2024-02-04 22:34:00
4,289-26-0537,04-09-2024 17:48,Y,Gionettitti,M,5,African American,Orthopedics,False,NaN,10,0,2024-09-04 17:48:00


In [9]:
# Standardize headers first so we don't have to type " Patient Age " with spaces
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('-', '_')

# Preview
print("--- NULL COUNT ---")
print(df.isnull().sum())
print("\n--- DATA TYPES ---")
print(df.dtypes)

--- NULL COUNT ---
patient_id                       0
patient_admission_date           0
patient_first_inital             0
patient_last_name                0
patient_gender                   0
patient_age                      0
patient_race                     0
department_referral           5400
patient_admission_flag           0
patient_satisfaction_score    6699
patient_waittime                 0
patients_cm                      0
date                             0
dtype: int64

--- DATA TYPES ---
patient_id                            object
patient_admission_date                object
patient_first_inital                  object
patient_last_name                     object
patient_gender                        object
patient_age                            int64
patient_race                          object
department_referral                   object
patient_admission_flag                  bool
patient_satisfaction_score           float64
patient_waittime                       int6

In [12]:
# Check for nulls
print(f"Null Dates: {df['date'].isnull().sum()}")
print("Sample Dates:", df['date'].head())
print(len(df))

Null Dates: 0
Sample Dates: 0   2024-03-20 08:47:00
1   2024-06-15 11:29:00
2   2024-06-20 09:13:00
3   2024-02-04 22:34:00
4   2024-09-04 17:48:00
Name: date, dtype: datetime64[ns]
9216


In [13]:
# Coerce errors will turn "garbage text" into NaT (Not a Time)
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Drop rows where date is NaT
df = df.dropna(subset=['date'])

print(f"Remaining Rows after Date cleanup: {len(df)}")

Remaining Rows after Date cleanup: 9216


In [14]:
print(f"Null Ages: {df['patient_age'].isnull().sum()}")
# Check for weird values
print("Min Age:", df['patient_age'].min())
print("Max Age:", df['patient_age'].max())

#No nulls in age, therefore no need to impute

Null Ages: 0
Min Age: 1
Max Age: 79


In [15]:
# all unique variations
print("Unique Genders:", df['patient_gender'].unique())
print("Null Genders:", df['patient_gender'].isnull().sum())

Unique Genders: ['M' 'F' 'NC']
Null Genders: 0


In [16]:
# 1. Fill Nulls
df['patient_gender'] = df['patient_gender'].fillna('NC')

# 2. Standardize (Title Case handles 'male' -> 'Male')
df['patient_gender'] = df['patient_gender'].str.title()

# 3. Map to simple format (Optional, depending on what you see in inspection)

gender_map = {'Male': 'M', 'Female': 'F', 'M': 'M', 'F': 'F', 'Nc': 'NC'}
df['patient_gender'] = df['patient_gender'].map(gender_map).fillna('NC')

print("Unique Genders after fix:", df['patient_gender'].unique())

Unique Genders after fix: ['M' 'F' 'NC']


In [17]:
print(f"Null Wait Times: {df['patient_waittime'].isnull().sum()}")

print(df['patient_waittime'].describe())

Null Wait Times: 0
count    9216.000000
mean       35.259874
std        14.735323
min        10.000000
25%        23.000000
50%        35.000000
75%        48.000000
max        60.000000
Name: patient_waittime, dtype: float64


In [18]:
print("Unique Depts:", df['department_referral'].unique())
print("Null Depts:", df['department_referral'].isnull().sum())

Unique Depts: [nan 'General Practice' 'Orthopedics' 'Gastroenterology' 'Physiotherapy'
 'Neurology' 'Cardiology' 'Renal']
Null Depts: 5400


In [19]:
# Fill NA with 'General Practice' or 'None'
df['department_referral'] = df['department_referral'].fillna('General Practice')

# Clean whitespace
df['department_referral'] = df['department_referral'].str.strip()

In [20]:


# 1. Shifts
df['arrival_hour'] = df['date'].dt.hour
def get_shift(hour):
    if 6 <= hour < 14: return 'Morning'
    elif 14 <= hour < 22: return 'Afternoon'
    else: return 'Night'
df['shift'] = df['arrival_hour'].apply(get_shift)

# 2. Age Groups
bins = [0, 18, 64, 150]
labels = ['Pediatric', 'Adult', 'Senior']
df['age_group'] = pd.cut(df['patient_age'], bins=bins, labels=labels)

# 3. Export
print("Final Check - Nulls per column:")
print(df.isnull().sum())



Final Check - Nulls per column:
patient_id                       0
patient_admission_date           0
patient_first_inital             0
patient_last_name                0
patient_gender                   0
patient_age                      0
patient_race                     0
department_referral              0
patient_admission_flag           0
patient_satisfaction_score    6699
patient_waittime                 0
patients_cm                      0
date                             0
arrival_hour                     0
shift                            0
age_group                        0
dtype: int64


In [22]:
print(f"Null Satisfaction Scores: {df['patient_satisfaction_score'].isnull().sum()}")
print(df['patient_satisfaction_score'].describe())

Null Satisfaction Scores: 6699
count    2517.000000
mean        4.992054
std         3.138043
min         0.000000
25%         2.000000
50%         5.000000
75%         8.000000
max        10.000000
Name: patient_satisfaction_score, dtype: float64


In [24]:
# 1. Fill Nulls with Median (Neutral assumption)
median_score = df['patient_satisfaction_score'].median()
df['patient_satisfaction_score'] = df['patient_satisfaction_score'].fillna(median_score)

# 2. Ensure they are Integers (scores are usually 1-10, not 5.5)
df['patient_satisfaction_score'] = df['patient_satisfaction_score'].astype(int)

#3. Upper and lower bounds
df['patient_satisfaction_score'] = df['patient_satisfaction_score'].clip(lower=0, upper=10)

print("Satisfaction Cleaned. Nulls:", df['patient_satisfaction_score'].isnull().sum())

Satisfaction Cleaned. Nulls: 0


In [26]:
df.head(20)

,patient_id,patient_admission_date,patient_first_inital,patient_last_name,patient_gender,patient_age,patient_race,department_referral,patient_admission_flag,patient_satisfaction_score,patient_waittime,patients_cm,date,arrival_hour,shift,age_group
0,145-39-5406,20-03-2024 08:47,H,Glasspool,M,69,White,General Practice,False,10,39,0,2024-03-20 08:47:00,8,Morning,Senior
1,316-34-3057,15-06-2024 11:29,X,Methuen,M,4,Native American/Alaska Native,General Practice,True,5,27,0,2024-06-15 11:29:00,11,Morning,Pediatric
2,897-46-3852,20-06-2024 09:13,P,Schubuser,F,56,African American,General Practice,True,9,55,0,2024-06-20 09:13:00,9,Morning,Adult
3,358-31-9711,04-02-2024 22:34,U,Titcombe,F,24,Native American/Alaska Native,General Practice,True,8,31,0,2024-02-04 22:34:00,22,Night,Adult
4,289-26-0537,04-09-2024 17:48,Y,Gionettitti,M,5,African American,Orthopedics,False,5,10,0,2024-09-04 17:48:00,17,Afternoon,Pediatric
5,255-51-2877,20-04-2023 00:13,H,Buff,M,58,Asian,General Practice,False,5,59,0,2023-04-20 00:13:00,0,Night,Adult
6,465-97-0990,23-08-2023 08:26,F,Perrat,F,68,White,General Practice,True,5,43,0,2023-08-23 08:26:00,8,Morning,Senior
7,157-31-7520,29-07-2023 16:57,K,Gwillim,F,47,Two or More Races,General Practice,True,5,23,0,2023-07-29 16:57:00,16,Afternoon,Adult
8,432-34-5614,19-02-2024 06:54,E,Dewhirst,F,79,White,General Practice,False,1,42,0,2024-02-19 06:54:00,6,Morning,Senior
9,609-17-8678,11-10-2024 05:25,M,Crebo,M,62,African American,General Practice,False,5,51,0,2024-10-11 05:25:00,5,Night,Adult


In [30]:
from google.colab import auth
import gspread
from google.auth import default

# 1. Authenticate
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 2. Connect to the Google Sheet

sheet_url = 'https://docs.google.com/spreadsheets/d/1T2iW_Ep9w0iieFiiTeK_efTsY84KiMCDJjPkhV7D9ew/edit?gid=0#gid=0'
workbook = gc.open_by_url(sheet_url)
worksheet = workbook.worksheet('ER_Data') # Must match tab name

# 3. Prepare Data for Upload
# Convert datetime objects to string format for JSON serialization
df_for_upload = df.copy()
df_for_upload['date'] = df_for_upload['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
data_to_upload = [df_for_upload.columns.values.tolist()] + df_for_upload.values.tolist()

# 4. Clear & Push
print("Clearing old data from cloud...")
worksheet.clear()

print(f"Pushing {len(df_for_upload)} rows to Google Sheets...")
worksheet.update('A1', data_to_upload)

print("✅ ETL Job Complete. Power BI can now be refreshed.")

Clearing old data from cloud...
Pushing 9216 rows to Google Sheets...


/tmp/ipython-input-1325317250.py:27: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data_to_upload)


✅ ETL Job Complete. Power BI can now be refreshed.


In [28]:
# --- DEBUGGING DATE ERRORS ---

print("1. Sample Raw Dates (Before Fix):")
print(df['date'].head(10))

# (Force Invalid Dates to NaT)
# errors='coerce' turns garbage text into NaT (Not a Time) instead of crashing
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Check how many rows failed
missing_dates = df['date'].isnull().sum()
print(f"\n2. Rows with Invalid Dates (Converted to NaT): {missing_dates}")

if missing_dates > 0:
    print("   Dropping these rows because we cannot analyze trends without a date.")
    df = df.dropna(subset=['date'])

print(f"\n3. Remaining Rows: {len(df)}")
print("   Date conversion successful.")

1. Sample Raw Dates (Before Fix):
0   2024-03-20 08:47:00
1   2024-06-15 11:29:00
2   2024-06-20 09:13:00
3   2024-02-04 22:34:00
4   2024-09-04 17:48:00
5   2023-04-20 00:13:00
6   2023-08-23 08:26:00
7   2023-07-29 16:57:00
8   2024-02-19 06:54:00
9   2024-10-11 05:25:00
Name: date, dtype: datetime64[ns]

2. Rows with Invalid Dates (Converted to NaT): 0

3. Remaining Rows: 9216
   Date conversion successful.
